In [6]:
export GOOGLE_APPLICATION_CREDENTIALS="E:\service\translator.json"

SyntaxError: invalid syntax (3080568155.py, line 1)

# app.py

In [5]:
import streamlit as st
from streamlit_webrtc import webrtc_streamer, AudioProcessorBase, WebRtcMode
from transformers import MarianMTModel, MarianTokenizer
import torch
from google.cloud import speech, texttospeech
from io import BytesIO
import numpy as np
import av

# Initialize Google APIs

In [4]:
speech_client = speech.SpeechClient()
tts_client = texttospeech.TextToSpeechClient()

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

# Load the Hugging Face translation model and tokenizer

In [ ]:
def load_translation_model(input_lang, output_lang):
    model_name = f"Helsinki-NLP/opus-mt-{input_lang}-{output_lang}"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    return model, tokenizer

# Define a custom audio processor for handling audio input from WebRTC

In [ ]:
class AudioProcessor(AudioProcessorBase):
    def __init__(self):
        self.transcript = None

    def recv_audio(self, frames, sample_rate):
        audio_data = np.frombuffer(frames.to_ndarray(), dtype=np.int16).tobytes()
        
        # Google Speech Recognition Config
        audio = speech.RecognitionAudio(content=audio_data)
        config = speech.RecognitionConfig(
            encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=sample_rate,
            language_code=input_language,
        )

        # Transcribe audio
        response = speech_client.recognize(config=config, audio=audio)
        self.transcript = " ".join([result.alternatives[0].transcript for result in response.results])
        return frames

# Frontend Configuration

In [ ]:
st.title("Real-Time Speech Translator with Hugging Face")
st.subheader("Convert Speech to Text, Translate, and Play Audio")

# Language Selection

In [ ]:
input_language = st.selectbox("Input Language", ["en", "es", "fr", "de"])
output_language = st.selectbox("Output Language", ["en", "es", "fr", "de"])

# Load the translation model based on selected languages

In [ ]:
translation_model, translation_tokenizer = load_translation_model(input_language, output_language)

# WebRTC Audio Streamer

In [ ]:
webrtc_ctx = webrtc_streamer(
    key="speech-to-text",
    mode=WebRtcMode.SENDONLY,
    audio_processor_factory=AudioProcessor,
    async_processing=True,
)

if webrtc_ctx.audio_processor and webrtc_ctx.audio_processor.transcript:
    transcript = webrtc_ctx.audio_processor.transcript
    st.write("Original Transcript:", transcript)

    # Hugging Face Translation Function
    def translate_text_huggingface(text):
        inputs = translation_tokenizer(text, return_tensors="pt", padding=True)
        translated_tokens = translation_model.generate(**inputs)
        translated_text = translation_tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        return translated_text

    # Translate the transcript
    translated_text = translate_text_huggingface(transcript)
    st.write("Translated Text:", translated_text)

    # Text-to-Speech
    def generate_audio(text, language_code):
        synthesis_input = texttospeech.SynthesisInput(text=text)
        voice = texttospeech.VoiceSelectionParams(language_code=language_code, ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)
        audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)
        response = tts_client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)
        return response.audio_content

    if st.button("Play Translated Audio"):
        audio_content = generate_audio(translated_text, output_language)
        st.audio(BytesIO(audio_content), format="audio/mp3")